In [1]:
# load python libraries
from dash import dcc, html, Dash, dash_table
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import numpy as np
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
import country_converter as coco
import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# read raw data
data = pd.read_csv("data/data_raw.csv")
# read reference table
ref_table = pd.read_csv("data/country_code_conversion.csv")

# clean relevant numeric columns in `data` and `ref_table`
data["Area Code (M49)"] = (
    data["Area Code (M49)"]
    .astype(str)
    .str.replace('"', '', regex=False)
    .str.strip()
    .astype("Int64")
)
ref_table["Numeric code"] = (
    ref_table["Numeric code"]
    .astype(str)
    .str.replace('"', '', regex=False)
    .str.strip()
    .astype("Int64")
)
# ref_table["Latitude (average)"] = (
#     ref_table["Latitude (average)"]
#     .astype(str)
#     .str.replace('"', '', regex=False)
#     .str.strip()
#     .astype("Float64")
# )
# ref_table["Longitude (average)"] = (
#     ref_table["Longitude (average)"]
#     .astype(str)
#     .str.replace('"', '', regex=False)
#     .str.strip()
#     .astype("Float64")
# )
ref_table["Alpha-3 code"] = (
    ref_table["Alpha-3 code"]
    .astype(str)
    .str.replace('"', '', regex=False)
    .str.strip()
)
data["Year"] = data["Year"].astype(int)

# fix inconsistency between raw data and reference table
data.loc[data["Area"] == "Sudan", "Area Code (M49)"] = 736

# left join `data` and `ref_table` using each area's M49 code
# so that latitude and longitude information can be included in `data`
data = data.merge(
    ref_table,
    how="left",
    left_on="Area Code (M49)",
    right_on="Numeric code"
)

# use coco library to tell which continent each area is located in
cc = coco.CountryConverter()
logging.getLogger("country_converter").setLevel(logging.ERROR)
data["Continent"] = cc.convert(names=data["Area Code (M49)"], to="continent", src="UNnumeric")
data["Continent"] = data["Continent"].replace("not found", pd.NA)

# filter out those area which cannot be matched with a continent
data = data[data['Continent'].notna()]
# select only relevant columns in `data`
data = data[['Area', 'Continent', #'Latitude (average)', 'Longitude (average)', 
             'Alpha-3 code', 'Year', 'Import', 'Export ',
             'Production', 'Consumption', 'Unit']]

# rename dirty column names
data = data.rename(columns={
    # 'Latitude (average)': 'Latitude',
    # 'Longitude (average)': 'Longitude',
    'Export ': 'Export',
    'Alpha-3 code': 'ISO-3'
})

# use grouping to eliminate duplicate rows
data = (data
    .groupby(['ISO-3', 'Year'])
    .agg({'Import': 'sum', 'Export': 'sum',
         'Production': 'sum', 'Consumption': 'sum'})
    .reset_index())

# calculate two derived columns: `Net Trade` and `Self-Sufficiency Ratio`
data['Net Trade'] = data['Export'] - data['Import']
data['Self-Sufficiency Ratio'] = data['Production'] / data['Consumption']

# display first 3 rows of `data`
data.head(3)

,ISO-3,Year,Import,Export,Production,Consumption,Net Trade,Self-Sufficiency Ratio
0,AFG,1993,30.0,0.0,0.0,30.0,-30.0,0.0
1,AFG,1994,90.0,0.0,0.0,90.0,-90.0,0.0
2,AFG,1995,120.0,0.0,0.0,120.0,-120.0,0.0


In [3]:
# display last 3 rows of `data`
data.tail(3)

,ISO-3,Year,Import,Export,Production,Consumption,Net Trade,Self-Sufficiency Ratio
5710,ZWE,2021,128.06,1183.12,22839.42,21784.36,1055.06,1.048432
5711,ZWE,2022,97.51,1749.37,26849.20,25197.34,1651.86,1.065557
5712,ZWE,2023,109.20,2222.53,23815.47,21702.14,2113.33,1.097379


In [4]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
load_figure_template("CERULEAN")
app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN, dbc_css])

app.layout = dbc.Container([
    dcc.Tabs(
        id="tabs",
        children=[
            dcc.Tab(
                    label="Global Overview",
                    value="tab1",
                    children = [
                        dbc.Row(html.H1(id="global-title",
                                        style={"text-align": "center"})),
                        dbc.Row([
                            dbc.Col([
                                dbc.Row(dbc.Card([
                                    dbc.Row(html.P("Select a metric below:")),
                                    dbc.Row(
                                        dcc.Dropdown(
                                            id="metric-picker",
                                            options=['Import', 'Export', 'Production', 'Consumption',
                                                     'Net Trade', 'Self-Sufficiency Ratio'],
                                            value="Import"
                                        ))],
                                    className="mb-3")),
                                dbc.Row(dbc.Card([
                                    dbc.Row(html.P("Select a year below:")),
                                    dbc.Row(
                                        dcc.Dropdown(
                                            id="year-picker",
                                            options=data['Year'].unique(),
                                            value=data['Year'].max()
                                        ))], className="h-100"))], 
                                    width=2,
                                    className="h-100"),
                            dbc.Col(
                                dbc.Card(
                                    dcc.Graph(id="global-map"),
                                    className="h-100"),
                                className="h-100"),
                            dbc.Col(
                                dbc.Card(
                                    dcc.Graph(id="global-time-series"),
                                    className="h-100"),
                                className="h-100")
                        ], align="stretch")]),
            dcc.Tab(
                label="TAB 2",
                value="tab2",
                children=[html.H1("Under Construction")])])])


@app.callback(
    Output('global-title', 'children'),
    Output('global-map', 'figure'),
    Output('global-time-series', 'figure'),
    Input("metric-picker", "value"),
    Input("year-picker", "value"),
)
def global_overview_plots(metric, year):
    
    title = f"Global Overview of Spice {metric}"
    
    global_map = px.choropleth(
        (data[data['Year'] == year]
            .sort_values('Year')),
        locations="ISO-3",
        color=metric,
        animation_frame="Year",
        locationmode="ISO-3"
    ).update_geos(fitbounds="locations").update_layout(
        margin={"r":0,"t":40,"l":0,"b":100}, 
        coloraxis_colorbar_x=.85
    ).update_layout(
        coloraxis=dict(cmin=data[data['Year'] == year][metric].quantile(0.03), 
                       cmax=data[data['Year'] == year][metric].quantile(0.97),
                       colorbar=dict(
                           orientation='h', xanchor='center', yanchor='top',
                           x=0.5, y=-0.15
                       )),
        title=dict(
            text=f"World Map of Spice {metric} in {year}",
            xanchor='center',
            yanchor='top',
            x=0.5
        )
    )

    global_time_series = px.choropleth(
        data,
        locations="ISO-3",
        color=metric,
        animation_frame="Year",
        locationmode="ISO-3",
        title=f"World Map of Spice {metric}"
    ).update_geos(fitbounds="locations").update_layout(
        margin={"r":0,"t":0,"l":0,"b":0}, 
        coloraxis_colorbar_x=.85
    )

    return title, global_map, global_time_series
    
if __name__ == "__main__":
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
